Tasks to be done for tree based model for model evaluation.

Load data from kaggle

merge tables to create final data

Calculating null percentages for each column

Seperate columns into categorical and numerical columns

Drop column with more than 90% of Null values

Fill null values according to type of data

Check if data is balanced or not

Add extra columns after manipulating existing columns

Add columns after aggreagtion of existing data

Removing outliers from the numerical data

Add identifier for the columns within range of 50 to 90% of null value so that tree based model handles with special care

Converting categorical data into numerical (encoding basically optional for case of Tree based data)

Perform above steps for test dataset as well

Create model using Lightgbm better as we have null values and good for handling imbalance data.

---------------
Load paysim model from kaggle
renaming columns in paysim_df
extracting new columns from existing columns and find common columns with cis-fraud dataset
merged both training dataset on basis of common columns and also added some extra columns in each
---------------

After that perform null handling in merged model and then perform normalisation of combined data and split into train and test split. 


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
import pandas as pd
df_train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
df_train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
df_test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
df_test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

In [3]:
train = pd.merge(df_train_transaction, df_train_identity, on='TransactionID', how='left')
test = pd.merge(df_test_transaction, df_test_identity, on='TransactionID', how='left')

In [4]:
import gc
del df_train_transaction, df_train_identity, df_test_transaction, df_test_identity
gc.collect()

0

In [5]:
paysim = pd.read_csv('/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv')
#print(f"IEEE train: {train.shape} | Test: {test.shape} | PaySim: {paysim.shape}")

In [7]:
for df in [train, test]:
    df['hour'] = (df['TransactionDT'] // 3600) % 24
    df['day']  = df['TransactionDT'] // (24 * 3600)

In [8]:
val_days = train['day'].max() - 30
val = train[train['day'] >= val_days].copy()
train_split = train[train['day'] < val_days].copy()

In [9]:
y_train = train_split['isFraud']
y_val   = val['isFraud']

In [10]:
print(f"Train period: day 0–{train_split['day'].max()} | Val period: day {val['day'].min()}–{val['day'].max()}")
print(f"Train fraud rate: {y_train.mean():.5f} | Val fraud rate: {y_val.mean():.5f}")

Train period: day 0–151 | Val period: day 152–182
Train fraud rate: 0.03501 | Val fraud rate: 0.03486


In [11]:
high_missing = train.isnull().mean()
cols_to_drop = high_missing[high_missing > 0.90].index.tolist() + ['TransactionID', 'TransactionDT', 'day']
train_split = train_split.drop(columns=cols_to_drop, errors='ignore')
val         = val.drop(columns=cols_to_drop, errors='ignore')
test        = test.drop(columns=cols_to_drop, errors='ignore')

In [12]:
cat_cols = [c for c in train_split.columns if train_split[c].dtype == 'object']

In [13]:
freq_features = ['card1','card2','card3','card4','card5','card6','addr1','addr2','P_emaildomain','R_emaildomain']
for col in freq_features:
    if col not in train_split.columns: continue
    freq_map = train_split[col].value_counts(normalize=True)
    for df in [train_split, val, test]:
        df[f'{col}_freq'] = df[col].map(freq_map).fillna(0)

In [14]:
for df in [train_split, val, test]:
    df['log_TransactionAmt'] = np.log1p(df['TransactionAmt'])
    df['Amt_centered'] = df['TransactionAmt'] - df['TransactionAmt'].mean()

In [15]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
encoders = {}
le = OneHotEncoder(handle_unknown='ignore', sparse=False)
for col in cat_cols:
    for df in [train_split,val,test]:
        if col in df.columns:
            df[col] = df[col].fillna('missing').astype('str')
    #le = LabelEncoder()
    '''
    
    train_split[col] = train_split[col].fillna('missing').astype(str)
    val[col] = val[col].fillna('missing').astype(str)
    if col in test.columns:
        test[col] = test[col].fillna('missing').astype(str)
        
    le.fit(train_split[[col]])
    train_encoded = pd.DataFrame(
        le.transform(train_split[[col]]),
        columns=le.get_feature_names_out([col]),
        index=train_split.index
    )

    val_encoded = pd.DataFrame(
        le.transform(val[[col]]),
        columns = le.get_feature_names_out([col]),
        index=val.index
    )

    

    train_split_enc = train_split.drop(col, axis=1).join(train_encoded)
    val_enc  = val.drop(col, axis=1).join(val_encoded)
    
    if col  in test.columns:
        test_encoded = pd.DataFrame(
            le.transform(test[[col]]),
            columns = le.get_feature_names_out([col]),
            index=test.index
            )
        test_enc  = test.drop(col, axis=1).join(test_encoded)
    
    
    
    
    train_split[col] = le.transform(train_split[[col]])
    val[col]   = le.transform(val[[col]])
    if col in test.columns:
        test[col]  = test[col].map(lambda x: len(le.categories_) if x not in le.categories_[0] else le.transform([[x]]).flatten())
    
    encoders[col] = le '''

In [16]:
train_split = train_split.fillna(-1)
val         = val.fillna(-1)
test        = test.fillna(-1)

In [17]:
X_train = train_split.drop('isFraud', axis=1)
X_val   = val.drop('isFraud', axis=1)
X_test  = test.copy()



In [18]:
for df in [X_train,X_val,X_test]:
    for col in df.select_dtypes(include=['category', 'object']).columns:
        df[col] = df[col].astype('category').cat.codes

In [27]:
from sklearn.metrics import average_precision_score, roc_curve
params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 256,
    'feature_fraction': 0.85,
    'bagging_fraction': 0.85,
    'bagging_freq': 5,
    'scale_pos_weight': (y_train==0).sum() / (y_train==1).sum(),
    'seed': 42,
    'verbose': -1
}

lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = cat_cols)
lgb_val   = lgb.Dataset(X_val,   y_val,categorical_feature = cat_cols,  reference=lgb_train)

model_v1 = lgb.train(
    params,
    lgb_train,
    num_boost_round=5000,
    valid_sets=[lgb_val],
    callbacks=[lgb.early_stopping(200), lgb.log_evaluation(100)]
)

val_pred = model_v1.predict(X_val)
print(f"\nSTAGE 1 — REAL IEEE VALIDATION AUC: {roc_auc_score(y_val, val_pred):.5f}")
pr_auc = average_precision_score(y_val, val_pred)

print("PR-AUC:", pr_auc)

fpr, tpr, thresholds = roc_curve(y_val, val_pred)
recall_at_3fpr = tpr[fpr <= 0.03].max()
print("Recall @ 3% FPR:", recall_at_3fpr)

Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.893356
[200]	valid_0's auc: 0.8942
[300]	valid_0's auc: 0.897352
[400]	valid_0's auc: 0.90202
[500]	valid_0's auc: 0.905069
[600]	valid_0's auc: 0.909397
[700]	valid_0's auc: 0.912936
[800]	valid_0's auc: 0.914013
[900]	valid_0's auc: 0.91478
[1000]	valid_0's auc: 0.916461
[1100]	valid_0's auc: 0.917801
[1200]	valid_0's auc: 0.919095
[1300]	valid_0's auc: 0.920849
[1400]	valid_0's auc: 0.920354
[1500]	valid_0's auc: 0.922302
[1600]	valid_0's auc: 0.922552
[1700]	valid_0's auc: 0.923015
[1800]	valid_0's auc: 0.923488
[1900]	valid_0's auc: 0.923951
[2000]	valid_0's auc: 0.924448
[2100]	valid_0's auc: 0.925576
[2200]	valid_0's auc: 0.92566
[2300]	valid_0's auc: 0.92634
[2400]	valid_0's auc: 0.926942
[2500]	valid_0's auc: 0.926933
Early stopping, best iteration is:
[2378]	valid_0's auc: 0.927144

STAGE 1 — REAL IEEE VALIDATION AUC: 0.92714
PR-AUC: 0.6185920726558336
Recall @ 3% FPR: 0.6679511881824021


In [28]:
import os
os.makedirs("/kaggle/working/models", exist_ok=True)
model_v1.save_model("/kaggle/working/models/fraud_datamodel_ieee.txt")

In [19]:
print("\n=== STAGE 2: Adding PaySim as a strong feature ===")

# Simple PaySim preprocessing
paysim = paysim.rename(columns={'step': 'hour'}) 
paysim['log_amount'] = np.log1p(paysim['amount']) #changes for scale adjustment
'''
paysim['type'] = paysim['type'].map({
    'CASH_OUT':0, 'TRANSFER':1, 'PAYMENT':2, 'CASH_IN':3, 'DEBIT':4
}).fillna(5)
'''

paysim['orig_balance_change'] = paysim['oldbalanceOrg'] - paysim['newbalanceOrig']
paysim['dest_balance_change'] = paysim['newbalanceDest'] - paysim['oldbalanceDest']
paysim['amount_balance_ratio'] = paysim['amount'] / (paysim['oldbalanceOrg'] + 1)


paysim['is_orig_balance_zero'] = (paysim['oldbalanceOrg'] == 0).astype(int)
paysim['is_dest_balance_zero'] = (paysim['oldbalanceDest'] == 0).astype(int)



paysim['orig_balance_error'] = (
    paysim['oldbalanceOrg'] - paysim['amount'] != paysim['newbalanceOrig']
).astype(int)

paysim['dest_balance_error'] = (
    paysim['oldbalanceDest'] + paysim['amount'] != paysim['newbalanceDest']
).astype(int)

paysim = paysim[[
    'hour',
    'amount',
    'log_amount',
    'type',
    #'oldbalanceOrg','newbalanceOrig',
    #'oldbalanceDest','newbalanceDest',
    'orig_balance_change','dest_balance_change',
    #'amount_balance_ratio',
    'is_orig_balance_zero','is_dest_balance_zero',
    #'orig_balance_error','dest_balance_error',
    'isFraud'
]]

paysim = pd.get_dummies(paysim, columns=['type'], drop_first=True)
#paysim_features = paysim[['hour','type','log_amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFraud']].fillna(-999)


=== STAGE 2: Adding PaySim as a strong feature ===


In [20]:
paysim_y = paysim['isFraud']
paysim_X = paysim.drop('isFraud', axis=1)

#splitting data on basis of time interval keeping split ratio 80-20
split_hour = paysim_X['hour'].quantile(0.8)

train_idx = paysim_X['hour'] <= split_hour
val_idx   = paysim_X['hour'] >  split_hour

X_train_paysim, X_val_paysim = paysim_X[train_idx], paysim_X[val_idx]
y_train_paysim, y_val_paysim = paysim_y[train_idx], paysim_y[val_idx]

print(set(X_train_paysim.index).intersection(set(X_val_paysim.index)))
#Handling class imbalance
scale_pos_weight_paysim = (y_train_paysim == 0).sum() / (y_train_paysim == 1).sum()
print(scale_pos_weight_paysim)
print(X_train_paysim.shape, X_val_paysim.shape)
print(X_train_paysim.shape)
print(X_train_paysim.columns)

set()
1289.4072672218017
(5113884, 11) (1248736, 11)
(5113884, 11)
Index(['hour', 'amount', 'log_amount', 'orig_balance_change',
       'dest_balance_change', 'is_orig_balance_zero', 'is_dest_balance_zero',
       'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT', 'type_TRANSFER'],
      dtype='object')


In [22]:
# Trying to build only paysim based model 
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
import lightgbm as lgb

params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "scale_pos_weight": scale_pos_weight_paysim
}

train_data = lgb.Dataset(X_train_paysim, label=y_train_paysim)
val_data   = lgb.Dataset(X_val_paysim, label=y_val_paysim)

model_ps = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    num_boost_round=500,
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
)




y_pred_paysim = model_ps.predict(X_val_paysim)

roc_auc = roc_auc_score(y_val_paysim, y_pred_paysim)
pr_auc  = average_precision_score(y_val_paysim, y_pred_paysim)

fpr, tpr, _ = roc_curve(y_val_paysim, y_pred_paysim)
recall_at_3fpr = tpr[fpr <= 0.03].max()

print("PaySim ROC-AUC:", roc_auc)
print("PaySim PR-AUC :", pr_auc)
print("Recall @ 3% FPR:", recall_at_3fpr)

'''
lgb.train(
    params,
    lgb_train,
    num_boost_round=5000,
    valid_sets=[lgb_val],
    callbacks=[lgb.early_stopping(200), lgb.log_evaluation(100)]
)
'''

[LightGBM] [Info] Number of positive: 3963, number of negative: 5109921
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.077649 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1234
[LightGBM] [Info] Number of data points in the train set: 5113884, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000775 -> initscore=-7.161938
[LightGBM] [Info] Start training from score -7.161938
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[10]	valid_0's auc: 0.928567
PaySim ROC-AUC: 0.9246128854898846
PaySim PR-AUC : 0.0883022146948342
Recall @ 3% FPR: 0.9094117647058824


'\nlgb.train(\n    params,\n    lgb_train,\n    num_boost_round=5000,\n    valid_sets=[lgb_val],\n    callbacks=[lgb.early_stopping(200), lgb.log_evaluation(100)]\n)\n'

In [24]:
import os
os.makedirs("/kaggle/working/models", exist_ok=True)
model_ps.save_model("/kaggle/working/models/fraud_datamodel_paysim.txt")

In [21]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
transfer_features =[
    'amount',
    'log_amount',
    'hour',
    'type_CASH_OUT',
    'type_TRANSFER',
    'type_PAYMENT',
    'type_DEBIT'
]

X_ps_transfer = paysim[transfer_features]
y_ps = paysim['isFraud']

split_hour = X_ps_transfer['hour'].quantile(0.8)
train_idx = X_ps_transfer['hour']<split_hour
val_idx = X_ps_transfer['hour']>split_hour
X_train_ps = X_ps_transfer[train_idx]
X_val_ps = X_ps_transfer[val_idx]

y_train_ps = y_ps[train_idx]
y_val_ps = y_ps[val_idx]

assert set(X_train_ps.index).isdisjoint(set(X_val_ps.index))

scale_pos_weight = (y_train_ps==0).sum()/(y_train_ps==1).sum()

params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "scale_pos_weight": scale_pos_weight
}

train_data = lgb.Dataset(X_train_ps, label=y_train_ps)
val_data   = lgb.Dataset(X_val_ps,   label=y_val_ps)

model_ps_transfer = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    num_boost_round=300,
    callbacks=[lgb.early_stopping(30)]
)

y_pred = model_ps_transfer.predict(X_val_ps)

roc_auc = roc_auc_score(y_val_ps, y_pred)
pr_auc  = average_precision_score(y_val_ps, y_pred)

fpr, tpr, _ = roc_curve(y_val_ps, y_pred)
recall_at_3fpr = tpr[fpr <= 0.03].max()

print("Transfer ROC-AUC:", roc_auc)
print("Transfer PR-AUC :", pr_auc)
print("Recall @ 3% FPR:", recall_at_3fpr)
#df_ieee['paysim_score'] = paysim_model.predict(X_ieee_for_paysim)

[LightGBM] [Info] Number of positive: 3955, number of negative: 5065142
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035742 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 719
[LightGBM] [Info] Number of data points in the train set: 5069097, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000780 -> initscore=-7.155157
[LightGBM] [Info] Start training from score -7.155157
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.915047
Transfer ROC-AUC: 0.9022465677537932
Transfer PR-AUC : 0.022695031994119038
Recall @ 3% FPR: 0.0


In [14]:
import os
os.makedirs("/kaggle/working/models", exist_ok=True)
model_ps_transfer.save_model("/kaggle/working/models/fraud_datamodel_paysim_transfer.txt")

In [22]:
# loading original ieee model 
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
ieee_model = lgb.Booster(model_file='/kaggle/working/models/fraud_datamodel_ieee.txt')

In [23]:
#loading original paysim model
paysim_model = lgb.Booster(model_file='/kaggle/working/models/fraud_datamodel_paysim.txt')

In [24]:
#loading paysim_transfer_model
paysim_model_transfer = lgb.Booster(model_file='/kaggle/working/models/fraud_datamodel_paysim_transfer.txt')

In [25]:
df_ieee = train
df_ieee['hour'] = df_ieee['TransactionDT'] // 3600
df_ieee['log_amount'] = np.log1p(df_ieee['TransactionAmt'])
df_ieee['amount'] = df_ieee['TransactionAmt']
df_ieee['type_TRANSFER'] = (df_ieee['ProductCD'] == 'W').astype(int)
df_ieee['type_CASH_OUT'] = (df_ieee['ProductCD'] == 'C').astype(int)
df_ieee['type_PAYMENT'] = (df_ieee['ProductCD'] == 'H').astype(int)
df_ieee['type_DEBIT']    = (df_ieee['ProductCD'] == 'R').astype(int)





In [27]:
transfer_features = [
    'amount','log_amount','hour',
    'type_CASH_OUT','type_TRANSFER',
    'type_PAYMENT','type_DEBIT'
]

df_ieee['paysim_score'] = paysim_model_transfer.predict(
    df_ieee[transfer_features]
)

In [28]:
X_train_final = X_train.copy()
X_val_final = X_val.copy()
X_train_final['paysim_score'] = df_ieee.loc[X_train_final.index,'paysim_score']
X_val_final['paysim_score'] = df_ieee.loc[X_val_final.index,'paysim_score']

hybrid_train_data = lgb.Dataset(X_train_final,label=y_train)
hybrid_val_data = lgb.Dataset(X_val_final,label=y_val)



In [29]:
params_ieee = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.03,
    'num_leaves': 64,
    'feature_fraction': 0.85,
    'bagging_fraction': 0.85,
    'bagging_freq': 1,
    'scale_pos_weight': (y_train==0).sum() / (y_train==1).sum(),
    'seed': 42,
    'verbose': -1
}

model_hybrid = lgb.train(
    params_ieee,
    hybrid_train_data,
    valid_sets=[hybrid_val_data],
    num_boost_round=800,
    callbacks=[lgb.early_stopping(50),lgb.log_evaluation(100)]
)

val_pred = model_hybrid.predict(X_val_final)
print(f"\nSTAGE 1 — REAL IEEE VALIDATION AUC: {roc_auc_score(y_val, val_pred):.5f}")
pr_auc = average_precision_score(y_val, val_pred)

print("PR-AUC:", pr_auc)

fpr, tpr, thresholds = roc_curve(y_val, val_pred)
recall_at_3fpr = tpr[fpr <= 0.03].max()
print("Recall @ 3% FPR:", recall_at_3fpr)

Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.90909
[200]	valid_0's auc: 0.918086
[300]	valid_0's auc: 0.919618
Early stopping, best iteration is:
[339]	valid_0's auc: 0.921102

STAGE 1 — REAL IEEE VALIDATION AUC: 0.92110
PR-AUC: 0.5421284389257255
Recall @ 3% FPR: 0.575465639049454


In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd

explainer = shap.TreeExplainer(ieee_model)
shap_values = explainer(X_val)

plt.savefig('kaggle/working/shap_force_fraud.png')
plt.show()

shap_df = pd.DataFrame(shap_values.values,columns= X_val.columns)
shap_df.to_csv('/kaggle/working/shap_values.csv',index=False)
print("SHAP evaluation complete. Check /kaggle/working/shap_summary.png for feature impacts.")

Everything fine till this point

In [2]:
!pip install shap